## Introduction
Before we can visualize the multi-omics data in CytoScape, we need to prepare data to be imported according to the unified database identifiers available.
Therefor, metabolite data is mapped to ChEBI IDS.

## Setup


In [ ]:
# check if libraries are already installed > otherwise install it
#if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
if(!"dplyr" %in% installed.packages()) BiocManager::install("dplyr")
if(!"BridgeDbR" %in% installed.packages()) BiocManager::install("BridgeDbR")
install.packages("rJava",repos = "http://cran.us.r-project.org" )
 ## See https://www.r-bloggers.com/2018/02/installing-rjava-on-ubuntu/ if you have issues with this package on Ubuntu.

#load libraries
library(dplyr)
library(BridgeDbR)
library(rJava)



## Process metabolite data


In [ ]:
setwd('..')

#Obtain data from step 8
mSet_CD <- read.csv("8-significantly_changed_metabolites_analysis/output/mbxData_CD.csv", na.strings=c("", "NA"))
mSet_UC <- read.csv("8-significantly_changed_metabolites_analysis/output/mbxData_UC.csv", na.strings=c("", "NA"))
#filter out unused columns
mSet_CD <- mSet_CD [,c(1:4)]
mSet_UC <- mSet_UC [,c(1:4)]

# Set Working Directory back to current folder
setwd("10-identifier_mapping")

##Retain all metabolite IDs from CD and UC for identifier mapping:
mSet_total <- unique(rbind(mSet_CD[,c(1,2)], mSet_UC[,c(1,2)]))

#Download the Metabolite mapping file (if it doesn't exist locally yet):
checkfile <- paste0(getwd(), '/' ,"data/metabolites.bridge")
if (!file.exists(checkfile)) {
  download.file("https://figshare.com/ndownloader/files/36197283", checkfile)
}
##Load the metabolite mapping file:
mapper <- BridgeDbR ::loadDatabase(checkfile)

## Obtain the System codes for the databases HMDB (source database of dataset) and ChEBI (intended output database)
code <- getOrganismCode("Homo sapiens")
code_mappingFrom <- getSystemCode("HMDB")
code_mappingTo   <- getSystemCode("ChEBI")

## Create a data frame with the mappings and the correct SystemCode
input = data.frame(
    source = rep(code_mappingFrom, length(mSet_total$HMDB_ID)),
    identifier = mSet_total$HMDB_ID)
#Obtain all mappings from HMDB to ChEBI
MultiMappings = BridgeDbR::maps(mapper, input, code_mappingTo)
#remove all rows in the mapped data which do not include the prefix "CHEBI"
MultiMappings <- MultiMappings %>% filter(grepl("CHEBI",mapping, fixed = TRUE))
#filter out double identifiers because there are one-to-many relationship between hmdb and chebi IDs
MultiMappings <- MultiMappings %>% distinct (MultiMappings$identifier, .keep_all = TRUE)
MultiMappings <- MultiMappings [,c(2,4)]

merged.data_CD<- merge(MultiMappings, mSet_CD,by.x="identifier", by.y="HMDB_ID",sort = TRUE, all.x = TRUE, all.y = TRUE)
merged.data_UC<- merge(MultiMappings, mSet_UC,by.x="identifier", by.y="HMDB_ID",sort = TRUE, all.x = TRUE, all.y = TRUE)
#filter out metabolites that has NA value for CHEBI
merged.data_CD<- merged.data_CD %>% tidyr::drop_na(mapping)
merged.data_UC<- merged.data_UC %>% tidyr::drop_na(mapping)
#filter out metabolites that has NA value for foldchange_disorder
merged.data_CD<- merged.data_CD %>% tidyr::drop_na(foldchange_disorder)
merged.data_UC<- merged.data_UC %>% tidyr::drop_na(foldchange_disorder)
#change column names
colnames(merged.data_CD) <- c("HMDBID","CHEBI", "label", "log2FC_met", "pvalue_met")
colnames(merged.data_UC) <- c("HMDBID","CHEBI", "label", "log2FC_met", "pvalue_met")



## Export the mapped metabolomics data:


In [ ]:
##Save the data file
write.table(merged.data_CD, 'output/mbx_mapped_data_CD.tsv', sep ="\t", row.names = FALSE)
write.table(merged.data_UC, 'output/mbx_mapped_data_UC.tsv', sep ="\t", row.names = FALSE)



##Print session info:


In [ ]:
##Print session info:
sessionInfo()



### Last, we create a Jupyter notebook and markdown file from this script


In [ ]:
#Jupyter Notebook file
if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
library(devtools)
library(rmd2jupyter)
rmd2jupyter("metabolomics_identifier_mapping.Rmd")
